In order for the notebook to work properly, you have to create folders in a way which is specified in the notebook later. If you create the necessary folders, copy csv file into the appropriate folder and run the notebook.
First folder's path should be like this: /content/drive/MyDrive/resume_app/

Second folder's path (database path) : content/drive/MyDrive/resume_app/database/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gradio --q --upgrade
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --q --upgrade
!pip install optimum --q --upgrade
!pip install langchain --q --upgrade
!pip install chromadb --q --upgrade
!pip install sentence-transformers --q --upgrade
!pip install langchain-community

In [ ]:
!pip install transformers --upgrade --q

In [ ]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.llms import HuggingFaceHub
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/resume_app/Resume.csv')
df = df['Resume_str']
df.to_csv('/content/drive/MyDrive/resume_app/Resume_modified.csv')
loader = CSVLoader(file_path='/content/drive/MyDrive/resume_app/Resume_modified.csv')

docs = loader.load()
len(docs)

2484

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

20478

In [ ]:
import numpy as np
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings

drive_path = "/content/drive/MyDrive/resume_app/database/"

embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

You need to uncomment the next cell if you run this notebook for the first time to create vector database in your Google Drive, and make sure to comment it after it has been created, because it takes several minutes to be created.

In [ ]:
# create the db for the first time and comment this section restart the kernel and run the next cell
#db = Chroma.from_documents(texts, embeddings, persist_directory=drive_path + "db")

In [ ]:
from langchain.vectorstores import Chroma

db = Chroma(persist_directory="content/drive/MyDrive/resume_app/database/db", embedding_function=embeddings)

In [ ]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

MODEL_NAME="mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

# Create a configuration for text generation based on the specified model name
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Set the maximum number of new tokens in the generated text to 1024.
# This limits the length of the generated output to 1024 tokens.
generation_config.max_new_tokens = 1024

# Set the temperature for text generation. Lower values (e.g., 0.0001) make output more deterministic, following likely predictions.
# Higher values make the output more random.
generation_config.temperature = 0.0001

# Set the top-p sampling value. A value of 0.95 means focusing on the most likely words that make up 95% of the probability distribution.
generation_config.top_p = 0.95

# Enable text sampling. When set to True, the model randomly selects words based on their probabilities, introducing randomness.
generation_config.do_sample = True

# Set the repetition penalty. A value of 1.15 discourages the model from repeating the same words or phrases too frequently in the output.
generation_config.repetition_penalty = 1.15


# Create a text generation pipeline using the initialized model, tokenizer, and generation configuration
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

# Create a LangChain pipeline that wraps the text generation pipeline and set a specific temperature for generation
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import torch

template = """
<s>[INST] <<SYS>>
Act as a Specialist in Human Resources. Use the resumes to choose most suitable candidates.
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain1 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

result = qa_chain1(
    "Can you recommend me someone who has more than 5 years experience in software development?"
)
print(result["result"].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Based on your requirements, I would suggest reviewing the following resumes:

1. John Smith - He has over 7 years of experience in software development and has worked with various programming languages such as Java, Python, and JavaScript.
2. Jane Doe - She has over 6 years of experience in software development and specializes in web development using technologies like HTML, CSS, and JavaScript.
3. Michael Lee - He has over 8 years of experience in software development and has expertise in full-stack development using technologies like React, Node.js, and MongoDB.
4. Sarah Kim - She has over 5 years of experience in software development and has worked on projects related to data analysis and machine learning using tools like R and Python.

These candidates have demonstrated their skills and experience in software development and could be suitable for your needs.


In [ ]:
conditional_template = """
<s>[INST] <<SYS>>
Act as a Specialist in Human Resources. You are helping the team choose most suitable candidate for job from the resumes.
<</SYS>>

{context}

If the candidate has at least 5 years of experience and has good English skills and also has driving license, only then can someone be considered as possible candidate.
{question} [/INST]
"""

conditional_prompt = PromptTemplate(template=conditional_template, input_variables=["context", "question"])

qa_chain2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": conditional_prompt},
)

conditional_result = qa_chain2(
    "We are looking for a person for the role of sous-chef, who is flexible. Can you recommend me 3 suitable candidates?"
)
print(conditional_result["result"].strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Based on your requirements, I would suggest the following three candidates:

1. John Smith - He has over 7 years of experience as a sous-chef and is fluent in both English and Spanish. He also holds a valid driver's license.
2. Jane Doe - She has been working as a sous-chef for over 6 years and has excellent communication skills in English. She also has a valid driver's license.
3. Michael Lee - He has over 4 years of experience as a sous-chef and is proficient in English. He also holds a valid driver's license.


In [ ]:
comparison_template = """
<s>[INST] <<SYS>>
Embark on a journey as an HR explorer comparing two candidates from the cosmic talent pool. Evaluate their skills, experiences, and potential contributions to guide the company's quest for excellence.
<</SYS>>

{context}

Compare candidate A and candidate B, then answer the following:
- In terms of technical expertise, who stands out and why?
- Considering interpersonal skills, who seems more adept at fostering teamwork?
- Which candidate exhibits a stronger alignment with the company's values?
{question}
[/INST]
"""

comparison_prompt = PromptTemplate(template=comparison_template, input_variables=["context", "question"])

qa_chain3 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": comparison_prompt},
)

comparison_result = qa_chain3(
    "Compare the candidates A and B in terms of their qualifications, skills, and cultural fit."
)
print(comparison_result["result"].strip())

In [ ]:
import gradio as gr

def generate_response(question):
    result = qa_chain1(question)
    return {"response": result["result"].strip()}

iface = gr.Interface(
    fn=generate_response,
    inputs="textbox",
    outputs="text",
    title="HR Specialist Chatbot",
    description="Ask any question related to selecting suitable candidates based on resumes.",
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c2b239d51616f307de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
